In [ ]:
# Сборка Android APK для "Изучение таблицы умножения" v2.3 с кэшированием

Этот notebook поможет собрать APK файл приложения для изучения таблицы умножения в Google Colab.

## Инструкция:
1. Запусти первые две ячейки (установка пакетов и подключение Google Drive)
2. Загрузи файлы learning_only.py, buildozer.spec, icon.png, star_empty.png, star_filled.png, click.wav, fail1.wav-fail5.wav и good1.wav-good5.wav в Colab
3. Запусти остальные ячейки по порядку
4. Скачай готовый APK файл

## Кэширование на Google Drive:
**Первая сборка** займет много времени (40-60 минут), но все последующие запуски будут **значительно быстрее**, так как скачанные и скомпилированные файлы будут браться из кэша на вашем диске.

## Особенности приложения v2.3:
- Линейное обучение от таблицы 2 до 9
- Примеры на умножение и деление в случайном порядке
- Система освоения: каждый пример нужно решить 5 раз подряд правильно
- Система очков: +1 за правильный ответ, -15 за неправильный/пропущенный, -10 за время
- Таймер на ответ (настраивается от 5 до 8 секунд)
- Кнопки Старт/Стоп для управления сессией
- Встроенная цифровая клавиатура для мобильных устройств
- Улучшенные прогресс-бары со звездочками (переносятся на новые строки)
- Повторение пройденных таблиц для закрепления
- Звуковой сигнал при появлении нового примера (click.wav)
- Случайные звуки ошибок при неправильных ответах (fail1.wav-fail5.wav)
- Мотивационные сообщения с позитивными звуками (good1.wav-good5.wav) - только в этапе изучения
- **НОВОЕ v2.3:** Этап "МАРАФОН" - набор 150 очков для перехода к следующей таблице
- **НОВОЕ v2.3:** Крупное отображение очков "Набрано XX из 150" во втором этапе
- **НОВОЕ v2.3:** Мотивация только в первом этапе, концентрация во втором
- **НОВОЕ v2.3:** Обновленные триггеры мотивации (25 подряд, прогресс изучения)
- Сохранение прогресса между сессиями


In [ ]:
# Установка необходимых пакетов
!apt update
!apt install -y git zip unzip openjdk-17-jdk python3-pip autoconf libtool pkg-config zlib1g-dev libncurses5-dev libncursesw5-dev libtinfo5 cmake libffi-dev libssl-dev

# Установка дополнительных библиотек для Kivy
!apt install -y libgl1-mesa-dev libgles2-mesa-dev libglu1-mesa-dev libxi-dev libxcursor-dev libxinerama-dev libxrandr-dev libxxf86vm-dev

# Установка Python пакетов
!pip install buildozer "cython>=3.0.0" 
!pip install kivy[base]


In [ ]:
# Подключение Google Drive для кэширования скачанных файлов
print("Подключаем ваш Google Drive для сохранения скачанных пакетов...")
from google.colab import drive
drive.mount('/content/drive')

# Настройка папок для кэша
import os

# Только скачанные файлы сохраняем на Google Drive (они не требуют прав выполнения)
GDRIVE_DOWNLOADS_DIR = '/content/drive/MyDrive/umnojenie_downloads_cache'
os.environ['BUILDOZER_DL_DIR'] = GDRIVE_DOWNLOADS_DIR

# Сборка происходит локально в Colab (быстрее и без проблем с правами)
os.environ['BUILDOZER_BUILD_DIR'] = '/tmp/buildozer_build'

# Создаем папки
os.makedirs(GDRIVE_DOWNLOADS_DIR, exist_ok=True)
os.makedirs('/tmp/buildozer_build', exist_ok=True)

print(f"Скачанные файлы сохраняются в: {GDRIVE_DOWNLOADS_DIR}")
print("Сборка происходит локально для избежания проблем с правами доступа")
print("Повторные запуски будут быстрее благодаря кэшу скачанных файлов!")


In [ ]:
# Создание структуры проекта
import os
os.makedirs('/content/app', exist_ok=True)
os.chdir('/content/app')
print("Рабочая директория:", os.getcwd())


In [ ]:
# Загрузка файлов проекта
from google.colab import files
print("Загрузи следующие файлы:")
print("1. learning_only.py (будет переименован в main.py)")
print("2. buildozer.spec")
print("3. icon.png")
print("4. star_empty.png (пустая звездочка для прогресс-бара)")
print("5. star_filled.png (заполненная звездочка для прогресс-бара)")
print("6. click.wav (звуковой файл для сигналов)")
print("7. fail1.wav, fail2.wav, fail3.wav, fail4.wav, fail5.wav (звуки ошибок)")
print("8. good1.wav, good2.wav, good3.wav, good4.wav, good5.wav (мотивационные звуки)")

uploaded = files.upload()

# Автоматическое исправление buildozer.spec для совместимости с Colab
if 'buildozer.spec' in uploaded:
    print("Исправляем buildozer.spec для совместимости с Colab...")
    
    with open('buildozer.spec', 'r', encoding='utf-8') as f:
        content = f.read()
    
    # Исправления для совместимости
    content = content.replace('python3==3.11.6', 'python3')  # Убираем точную версию
    content = content.replace('learning_only.py', 'main.py')  # Исправляем имя файла
    content = content.replace('cython==3.0.10', '')  # Убираем cython из requirements
    
    # Убираем лишние запятые
    content = content.replace('kivy,pyjnius==1.6.1,', 'kivy,pyjnius==1.6.1')
    content = content.replace('kivy,pyjnius==1.6.1,cython==3.0.10', 'kivy,pyjnius==1.6.1')
    
    with open('buildozer.spec', 'w', encoding='utf-8') as f:
        f.write(content)
    
    print("✓ buildozer.spec исправлен")

# Переименование learning_only.py в main.py если нужно
if 'learning_only.py' in uploaded:
    import shutil
    shutil.move('learning_only.py', 'main.py')
    print("✓ learning_only.py переименован в main.py")

# Проверка наличия звуковых файлов
if 'click.wav' not in uploaded:
    print("⚠️ Внимание: click.wav не загружен. Звуковые сигналы работать не будут.")
    print("Создаем пустой файл для избежания ошибок...")
    with open('click.wav', 'wb') as f:
        f.write(b'')

# Проверка звуков ошибок
fail_sounds = ['fail1.wav', 'fail2.wav', 'fail3.wav', 'fail4.wav', 'fail5.wav']
missing_fail_sounds = [sound for sound in fail_sounds if sound not in uploaded]

if missing_fail_sounds:
    print(f"⚠️ Внимание: не загружены звуки ошибок: {', '.join(missing_fail_sounds)}")
    print("Создаем пустые файлы для избежания ошибок...")
    for sound in missing_fail_sounds:
        with open(sound, 'wb') as f:
            f.write(b'')
else:
    print("✓ Все звуковые файлы ошибок загружены!")

# Проверка мотивационных звуков (НОВОЕ в v2.2)
good_sounds = ['good1.wav', 'good2.wav', 'good3.wav', 'good4.wav', 'good5.wav']
missing_good_sounds = [sound for sound in good_sounds if sound not in uploaded]

if missing_good_sounds:
    print(f"⚠️ Внимание: не загружены мотивационные звуки: {', '.join(missing_good_sounds)}")
    print("Создаем пустые файлы для избежания ошибок...")
    for sound in missing_good_sounds:
        with open(sound, 'wb') as f:
            f.write(b'')
else:
    print("✓ Все мотивационные звуки загружены!")


In [ ]:
# Проверка файлов
print("Файлы в проекте:")
for file in os.listdir('.'):
    size = os.path.getsize(file) if os.path.isfile(file) else 'папка'
    print(f"- {file} ({size} bytes)" if size != 'папка' else f"- {file} (папка)")

# Проверяем, что все необходимые файлы есть
required_files = ['main.py', 'buildozer.spec', 'icon.png']
star_files = ['star_empty.png', 'star_filled.png']
sound_files = ['click.wav', 'fail1.wav', 'fail2.wav', 'fail3.wav', 'fail4.wav', 'fail5.wav']
good_sound_files = ['good1.wav', 'good2.wav', 'good3.wav', 'good4.wav', 'good5.wav']

missing_files = [f for f in required_files if not os.path.exists(f)]
missing_stars = [f for f in star_files if not os.path.exists(f)]
missing_sounds = [f for f in sound_files if not os.path.exists(f)]
missing_good_sounds = [f for f in good_sound_files if not os.path.exists(f)]

if missing_files:
    print(f"\n❌ Отсутствуют обязательные файлы: {', '.join(missing_files)}")
    print("Загрузите недостающие файлы перед продолжением!")
elif missing_stars:
    print(f"\n⚠️ Отсутствуют файлы звездочек: {', '.join(missing_stars)}")
    print("Прогресс-бар будет показывать текстовые звездочки вместо изображений")
    print("✅ Основные файлы загружены, можно продолжать!")
elif missing_sounds or missing_good_sounds:
    all_missing = missing_sounds + missing_good_sounds
    print(f"\n⚠️ Отсутствуют звуковые файлы: {', '.join(all_missing)}")
    print("Приложение будет работать, но без некоторых звуков")
    print("✅ Основные файлы загружены, можно продолжать!")
else:
    print("\n✅ Все файлы загружены (включая звездочки и звуки)!")
    print("⭐ Изображения звездочек: star_empty.png, star_filled.png")
    print("🎵 Звуки ошибок: fail1-fail5.wav")
    print("🎉 Мотивационные звуки: good1-good5.wav")


In [ ]:
# Сборка APK v2.3 (это займёт 20-40 минут)
print("Начинаем сборку APK v2.3...")
print("Это займёт много времени (20-40 минут), будь терпелив!")
print("\nОсобенности приложения v2.3:")
print("- Система очков и освоения примеров")
print("- Встроенная цифровая клавиатура")
print("- Настраиваемый таймер (5-8 секунд)")
print("- Улучшенные прогресс-бары со звездочками (переносятся на строки)")
print("- Сохранение прогресса")
print("- Звуковые сигналы при новых примерах")
print("- Случайные звуки ошибок (5 разных)")
print("- Мотивационные сообщения только в этапе изучения")
print("- НОВОЕ v2.3: Этап МАРАФОН - набор 150 очков!")
print("- НОВОЕ v2.3: Крупное отображение очков во втором этапе")
print("- НОВОЕ v2.3: Концентрация в марафоне (без отвлекающих popup)")
print("- НОВОЕ v2.3: Обновленные триггеры мотивации")
print("\nНачинаем сборку...")
!buildozer android debug


In [ ]:
# Проверка результата и скачивание APK
import glob

apk_files = glob.glob('bin/*.apk')
if apk_files:
    apk_file = apk_files[0]
    print(f"✅ APK файл создан: {apk_file}")
    
    # Получаем размер файла
    size_mb = os.path.getsize(apk_file) / (1024 * 1024)
    print(f"Размер APK: {size_mb:.1f} MB")
    
    # Скачивание APK
    files.download(apk_file)
    print("\n🎉 APK файл v2.3 готов к установке на Android устройство!")
    print("\nВозможности приложения v2.3:")
    print("✓ Изучение таблиц умножения от 2 до 9")
    print("✓ Система очков и освоения примеров")
    print("✓ Встроенная цифровая клавиатура")
    print("✓ Настраиваемый таймер (5-8 секунд)")
    print("✓ Улучшенные прогресс-бары со звездочками (переносятся на строки)")
    print("✓ Сохранение прогресса между сессиями")
    print("✓ Звуковые сигналы при новых примерах")
    print("✓ Случайные звуки ошибок (5 разных файлов)")
    print("✓ Мотивационные сообщения с позитивными звуками (только в этапе изучения)")
    print("✓ НОВОЕ v2.3: Двухэтапная система обучения:")
    print("   • Этап 1: Изучение с мотивацией и звездочками")
    print("   • Этап 2: МАРАФОН - набор 150 очков для перехода к следующей таблице")
    print("✓ НОВОЕ v2.3: Крупное отображение 'Набрано XX из 150' во втором этапе")
    print("✓ НОВОЕ v2.3: Концентрация в марафоне (без отвлекающих popup)")
    print("✓ НОВОЕ v2.3: Система достижений только в первом этапе:")
    print("   • Серия из 25 правильных ответов подряд")
    print("   • Этапы изучения (25%, 50%, 75%)")
    print("✓ НОВОЕ v2.3: Подробное объяснение правил марафона")
    print("✓ Повторение пройденного материала")
else:
    print("❌ APK файл не найден. Проверьте ошибки сборки выше.")
    
# Показать все файлы в bin/
if os.path.exists('bin'):
    print("\nФайлы в папке bin/:")
    for file in os.listdir('bin'):
        size_mb = os.path.getsize(f'bin/{file}') / (1024 * 1024)
        print(f"- {file} ({size_mb:.1f} MB)")
